In [17]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sf-booking/hotels_test.csv
/kaggle/input/sf-booking/hotels_train.csv
/kaggle/input/sf-booking/submission.csv


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn import preprocessing


зафиксируем RANDOM_SEED, чтобы эксперименты были воспроизводимы

In [19]:
RANDOM_SEED = 42

Подгрузим наши данные из соревнования

In [20]:
train = pd.read_csv('/kaggle/input/sf-booking/hotels_train.csv') # датасет для обучения
test = pd.read_csv('/kaggle/input/sf-booking/hotels_test.csv') # датасет для предсказания
sample_submission = pd.read_csv('/kaggle/input/sf-booking/submission.csv') # самбмишн

объединяем датасеты в один для корректной работы

In [21]:
test['reviewer_score'] = 0 # в тесте у нас нет значения reviewer_score, мы его должны предсказать, по этому пока просто заполняем нулями
train['sample'] = 1 # помечаем где у нас трейн
test['sample'] = 0 # помечаем где у нас тест
data = test.append(train, sort=False).reset_index(drop=True)

In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515738 entries, 0 to 515737
Data columns (total 18 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               515738 non-null  object 
 1   additional_number_of_scoring                515738 non-null  int64  
 2   review_date                                 515738 non-null  object 
 3   average_score                               515738 non-null  float64
 4   hotel_name                                  515738 non-null  object 
 5   reviewer_nationality                        515738 non-null  object 
 6   negative_review                             515738 non-null  object 
 7   review_total_negative_word_counts           515738 non-null  int64  
 8   total_number_of_reviews                     515738 non-null  int64  
 9   positive_review                             515738 non-null  object 
 

lat — географическая широта отеля и 
lng — географическая долгота отеля с пропусками.
логично заменить их нулем


In [23]:
data['lat'] = data['lat'].fillna(0, inplace=True)
data['lng'] = data['lng'].fillna(0, inplace=True)

сколько уникальных отелей?

ПОЛНЫЙ СПИСОК 25 ЛУЧШИХ ОТЕЛЕЙ МИРА 2023 ГОДА. https://www.acslocks.com/top-100-best-hotels-in-the-world/ Создаем дополнительный признак 'best', равный 1 если отель входит в этот список.

In [24]:
trip_best = ['Capella Ubud','Mardan Palace','Burj Al Arab','Amparo','Monastero Santa Rosa Hotel & Spa','Hôtel 71','Inkaterra Hacienda Urubamba',
               'Head over Hills','Ritz-Carlton','Sunset Key Cottages','Fogo Island Inn','Waldorf Astoria Maldives Ithaafushi','El Fenn Marrakech',
               'Atlantis Paradise','Londolozi','Angama Mara','RAAS Jodhpur','Archer Hotel Austin','Emirates Palace','Plaza','Westin Excelsior',
               'Cotton House','Palms Casino Resort','Lime Wood','Ett Hem','Hotel du Cap-Eden-Roc','Four Seasons Hotel Gresham Palace',
               'Canaves Oia Epitome','Secret Bay','Raffles','Areias do Seixo','Awasi Patagonia','Belmond Jimbaran Puri','Singita Kruger National Park',
               'Katikies Santorini','Amangani','Armani Hotel Dubai','The Sukhothai Bangkok','The Mulia Bali','Oyster Box','Anantara Chiang Mai Resort & Spa',
               'Baur au Lac','Jamaica Inn','The Oberai','St.Regis Florence by Marriott','The Leela Palace Udaipur','Manoir Hovey','Belmond Charleston Place',
               'Deer Path Inn','The Oberoi Rajvilas','Badrutt’s Palace Hotel','Royal Mansour','Necker Island','Hotel cafe royal','C Lazy U Ranch',
               'Explora Valle Sagrado','The Peninsula Shanghai','Cala de Mar Resort and Spa Ixtapa','La Residence Franschhoek','The Silo','Rabbit Hill Inn',
               'The Lodge at Blue Sky','Rambagh Palace','The Mark','The Oberoi Amarvilas','Mandarin Oriental','The Reverie Saigon','Nayara Gardens',
               'Hoshinoya Tokyo','Sol y Luna','Casa Chameleon at Las Catalinas','The Georges','The Middle House','Kamalame cay','Katikies Mykonos',
               'Sonnenalp Hotel','Bisate Lodge','The Wickaninnish Inn','Florblanca Resort','Rosewood Mayakoba','China’sodod Beijing','The Temple House',
               'Santa Monica Proper Hotel','Le Barthélemy Hotel & Spa','Six Senses Douro Valley','Turtle Island Resort','II San Pietro di Positano',
               'Grand Hotel Tremezzo','Gibb’s Farm','COMO The Treasury','La Peer Hotel','Explora Rapa Nui','Kasbah Tamadot','Frangipani Beach Resort',
               'Cap Rocat','The Milestone Hotel & Residences','Cavas Wine Lodge','Rancho Valencia Resort & Spa','The Boulders','Tulemar Bungalows and Villas- Manuel Antonio']

In [25]:
def rating(name):
    name = name.split()[:2]
    if len(name) > 1:
        x = name[0] +' ' + name[1]
    else:
        x = name[0]
    if x in trip_best:
        return 1
    return 0
data['best'] = data['hotel_name'].apply(rating)

проанализируем дату. преобразуем её

In [26]:
data['date'] = pd.to_datetime(data['review_date'])

посмотрим самые ранний и самый поздний отзывы

In [27]:
data['date'].min()

Timestamp('2015-08-04 00:00:00')

In [28]:
data['date'].max()

Timestamp('2017-08-03 00:00:00')

у нас отзывы за 2 года. последние отзывы более актуальны. разобъем на 3 периода по степени значимости: месяц, полгода, год, 2 года

In [29]:
time0 = '2017-07-03'
time1 = '2017-02-03'
time2 = '2016-08-03'
def period(time_otz):
    time_otz = str(time_otz)
    if time_otz > time0:
        return 0
    elif time_otz > time1:
        return 1
    elif time_otz > time2:
        return 2
    return 3

data['time_act'] = data['date'].apply(period)

кодируем национальность автора отзыва

In [30]:
import category_encoders as ce # импорт для работы с кодировщиком
bin_encoder = ce.BinaryEncoder(cols=['reviewer_nationality']) # указываем столбец для кодирования
type_bin = bin_encoder.fit_transform(data['reviewer_nationality'])
data = pd.concat([data, type_bin], axis=1)

разница между отрицательными и положительными отзывами (количество слов)

In [31]:
data['razn'] = data['review_total_positive_word_counts'] - data['review_total_negative_word_counts']

Из тегов выясним, на сколько ночей останавливаются путешественники в отелях

In [32]:
def get_stayed_nights(tags):
    tags_list = tags.split(' ')
    if 'Stayed' in tags_list:
        night_index = tags_list.index('Stayed') + 1
        return int(tags_list[night_index])
    else:
        return np.NaN

data['stayed_nights'] = data['tags'].apply(get_stayed_nights)

Заменим пропуски средним

In [33]:
data['stayed_nights'] = data['stayed_nights'].fillna(int(data['stayed_nights'].mean()))

убираем признаки которые еще не успели обработать

In [34]:
object_columns = [s for s in data.columns if data[s].dtypes == 'object' or data[s].dtypes == 'datetime64[ns]']
data.drop(object_columns, axis = 1, inplace=True)

Разделим объединенный датасет на трейн и тестовую часть 

In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515738 entries, 0 to 515737
Data columns (total 20 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   additional_number_of_scoring                515738 non-null  int64  
 1   average_score                               515738 non-null  float64
 2   review_total_negative_word_counts           515738 non-null  int64  
 3   total_number_of_reviews                     515738 non-null  int64  
 4   review_total_positive_word_counts           515738 non-null  int64  
 5   total_number_of_reviews_reviewer_has_given  515738 non-null  int64  
 6   reviewer_score                              515738 non-null  float64
 7   sample                                      515738 non-null  int64  
 8   best                                        515738 non-null  int64  
 9   time_act                                    515738 non-null  int64  
 

In [36]:
train = data.query('sample == 1').drop(['sample'], axis=1)
test = data.query('sample == 0').drop(['sample'], axis=1)

y = train.reviewer_score.values # наш таргет
X = train.drop(['reviewer_score'], axis=1) # трейн

Воспользуемся специальной функцией train_test_split для разбивки тестовых данных,
выделим 20% данных на валидацию (параметр test_size)

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

проверяем размеры

In [38]:
test.shape, train.shape, X.shape, X_train.shape, X_test.shape

((128935, 19), (386803, 19), (386803, 18), (309442, 18), (77361, 18))

Создаём модель

In [39]:
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

Обучаем модель на тестовом наборе данных (который часть трейна)

In [40]:
model.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   36.6s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.4min finished


RandomForestRegressor(n_jobs=-1, random_state=42, verbose=1)

Используем обученную модель для предсказания рейтинга ресторанов в тестовом наборе данных (который часть трейна)
Предсказанные значения записываем в переменную y_pred

In [41]:
y_pred = model.predict(X_test)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.7s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.7s finished


Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
Метрика называется Mean Absolute Percentage Error (MAPE) и показывает среднее абсолютное отклонение в процентах предсказанных значений от фактических

Function for MAPE

In [42]:
def MAPE(y_test,y_pred):
    mape = np.mean(np.abs((y_test - y_pred)/y_test))*100
    return mape


print('MAPE:', round(MAPE(y_test, y_pred), 5))

MAPE: 13.71152


Удаляем "лишний" признак у тестовой выборки (которая "настоящая")

In [43]:
test = test.drop(['reviewer_score'], axis=1)

Используем обученную модель для предсказания рейтинга 
Предсказанные значения записываем в переменную predict_submission

In [44]:
predict_submission = model.predict(test)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    1.1s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    2.6s finished


Округляем результат до сотых долей

In [45]:
for i in range(len(predict_submission)):
    predict_submission[i] = round(predict_submission[i], 2)

predict_submission

array([8.15, 7.27, 7.66, ..., 8.12, 9.37, 7.06])

Сохраняем результат в файл сабмишна

In [46]:
sample_submission['reviewer_score'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head()

,reviewer_score,id
0,8.15,488440
1,7.27,274649
2,7.66,374688
3,9.40,404352
4,9.35,451596


Краткая статистика

In [47]:
sample_submission.describe()

,reviewer_score,id
count,128935.000000,128935.000000
mean,8.342456,258065.135533
std,1.122012,148917.547662
min,3.220000,2.000000
25%,7.680000,129124.000000
50%,8.600000,257459.000000
75%,9.230000,387599.000000
max,10.000000,515737.000000
